# Project Title: Data scientist salary prediction 

In [1]:
#mporting libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from statsmodels.tools.eval_measures import rmse
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import random

In [2]:
# Uploading dataset
salary=  df= pd.read_csv("data_cleaned_2021.csv")

# Handling missing values

In [3]:
#Replacing object and int values of -1 with NaN
salary=salary.replace(['-1'],[np.nan])
salary=salary.replace([-1],[np.nan])
salary = salary.drop("index",axis=1)

In [4]:
salary["Competitors"] = salary["Competitors"].fillna(salary["Competitors"].mode()[0]) 
salary["Industry"] = salary["Industry"].fillna(salary["Industry"].mode()[0]) 
salary["Sector"] = salary["Sector"].fillna(salary["Sector"].mode()[0]) 
salary["Headquarters"] = salary["Headquarters"].fillna(salary["Headquarters"].mode()[0]) 
salary["Rating"] = salary["Rating"].fillna(salary["Rating"].mean())
salary["Age"] = salary["Age"].fillna(salary["Age"].median())
salary["Founded"] = salary["Founded"].fillna(salary["Founded"].median())
salary["Company Name"]=salary["Company Name"].str[:-4] #"company Name column conatin name with rating also so we are removing rating given in that column"
salary.drop("company_txt",axis=1,inplace=True)

## There are no missing values in data frame

In [5]:
#Here we made new column derived for take out state abrevation
salary["Location_abb"]=salary["Location"].str[-2:]

In [6]:
#Here we checking derived abbrevation of state and present column named as job location
salary["Check"]=salary["Job Location"].equals(salary["Location_abb"])

In [7]:
salary["Check"].value_counts() #Here we get to know that job location column is not useful so we are drop that column

True    742
Name: Check, dtype: int64

We dropped "location" column as abbrevation same

In [8]:
salary.drop("Location",axis=1,inplace=True) 
salary.drop("Location_abb",axis=1,inplace=True)
salary.drop("Check",axis=1,inplace=True) # Here we drop location as well as location_abb and check column also

In [9]:
salary.loc[salary['Age'] < 14 , 'Age'] = random.randint(14,67)

In [10]:
salary.loc[salary['Age'] > 67 , 'Age'] = random.randint(14,67)

In [12]:
salary.drop(["Company Name","Upper Salary","Lower Salary","Employer provided","Hourly","Industry","Salary Estimate","Job Title","Competitors","Headquarters","Job Description"],axis=1,inplace=True)

**Dummy encode the catergorical variables**

In [13]:
df_categoric = salary.select_dtypes(include = object)
dummy_encoded_variables = pd.get_dummies(df_categoric, drop_first = True)

In [14]:
dummy_encoded_variables.head()

,Size_10000+,Size_1001 - 5000,Size_201 - 500,Size_5001 - 10000,Size_501 - 1000,Size_51 - 200,Size_unknown,Type of ownership_Company - Private,Type of ownership_Company - Public,Type of ownership_Government,...,job_title_sim_data modeler,job_title_sim_data scientist,job_title_sim_director,job_title_sim_machine learning engineer,job_title_sim_na,job_title_sim_other scientist,seniority_by_title_na,seniority_by_title_sr,Degree_P,Degree_na
0,0,0,0,0,1,0,0,1,0,0,...,0,1,0,0,0,0,1,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,0,0,0,0,1,0,0,1,0,0,...,0,1,0,0,0,0,1,0,0,0
3,0,1,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,1,0,0,1
4,0,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,1,0,0,1


In [15]:
dummy_encoded_variables.shape

(742, 99)

**Concatenate numerical and dummy encoded categorical variables**

In [16]:
df_numeric = salary.select_dtypes(include=np.number)
# concatenate the numerical and dummy encoded categorical variables column-wise
df_dummy = pd.concat([df_numeric, dummy_encoded_variables], axis=1)

# display data with dummy variables
df_dummy.head()

,Rating,Founded,Avg Salary(K),Age,Python,spark,aws,excel,sql,sas,...,job_title_sim_data modeler,job_title_sim_data scientist,job_title_sim_director,job_title_sim_machine learning engineer,job_title_sim_na,job_title_sim_other scientist,seniority_by_title_na,seniority_by_title_sr,Degree_P,Degree_na
0,3.8,1973.0,72.0,48.0,1,0,0,1,0,1,...,0,1,0,0,0,0,1,0,0,0
1,3.4,1984.0,87.5,37.0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,4.8,2010.0,85.0,23.0,1,1,0,1,1,1,...,0,1,0,0,0,0,1,0,0,0
3,3.8,1965.0,76.5,56.0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1
4,2.9,1998.0,114.5,23.0,1,0,0,1,1,1,...,0,1,0,0,0,0,1,0,0,1


In [17]:
# add the intercept column using 'add_constant()'
df_dummy = sm.add_constant(df_dummy)

# separate the independent and dependent variables
# drop(): drops the specified columns
X = df_dummy.drop(["Avg Salary(K)"], axis = 1)

# extract the target variable from the data set
y = df_dummy[["Avg Salary(K)"]]


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)
print("The shape of X_train is:",X_train.shape)
print("The shape of X_test is:",X_test.shape)
print("The shape of y_train is:",y_train.shape)
print("The shape of y_test is:",y_test.shape)

The shape of X_train is: (593, 119)
The shape of X_test is: (149, 119)
The shape of y_train is: (593, 1)
The shape of y_test is: (149, 1)


# 1.OLS

In [19]:
linreg_full = sm.OLS(y_train, X_train).fit()
print(linreg_full.summary())

                            OLS Regression Results                            
Dep. Variable:          Avg Salary(K)   R-squared:                       0.665
Model:                            OLS   Adj. R-squared:                  0.583
Method:                 Least Squares   F-statistic:                     8.064
Date:                Tue, 29 Mar 2022   Prob (F-statistic):           9.91e-62
Time:                        10:16:09   Log-Likelihood:                -2666.4
No. Observations:                 593   AIC:                             5569.
Df Residuals:                     475   BIC:                             6086.
Df Model:                         117                                         
Covariance Type:            nonrobust                                         
                                                       coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------

In [20]:
linreg_full_predictions = linreg_full.predict(X_test)
linreg_full_predictions

273    150.177478
414     52.980414
529    119.687484
502     96.027582
729     84.471525
          ...    
247    104.337667
341    103.392522
298    136.500482
132     90.253924
656    129.297354
Length: 149, dtype: float64

In [21]:
actual_salary = y_test["Avg Salary(K)"]
actual_salary

273     90.0
414     62.5
529    125.0
502     97.5
729     70.0
       ...  
247     50.0
341    111.5
298    162.5
132     81.5
656    114.0
Name: Avg Salary(K), Length: 149, dtype: float64

In [22]:
linreg_full_rmse = rmse(actual_salary,linreg_full_predictions )
linreg_full_rsquared = linreg_full.rsquared
linreg_full_rsquared_adj = linreg_full.rsquared_adj 

In [23]:
cols = ['Model', 'RMSE', 'R-Squared', 'Adj. R-Squared','Accuracy']
result_tabulation = pd.DataFrame(columns = cols)
linreg_full_metrics = pd.Series({'Model': "Linreg full model ",
                     'RMSE':linreg_full_rmse,
                     'R-Squared': linreg_full_rsquared,
                     'Adj. R-Squared': linreg_full_rsquared_adj     
                   })
result_tabulation = result_tabulation.append(linreg_full_metrics, ignore_index = True)
result_tabulation

C:\Users\OMKAR\AppData\Local\Temp/ipykernel_16636/2469086403.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_tabulation = result_tabulation.append(linreg_full_metrics, ignore_index = True)


,Model,RMSE,R-Squared,Adj. R-Squared
0,Linreg full model,28.33744,0.665127,0.582642


# 2.Linear Regression with SGD

In [24]:
# importing library min max scaler to scale data
from sklearn.preprocessing import MinMaxScaler
#import library for implement Linear Regression
from sklearn.linear_model import SGDRegressor 
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.preprocessing import StandardScaler


In [25]:
scaler= StandardScaler()

In [26]:
scaler.fit(X)

StandardScaler()

In [27]:
X1= scaler.fit_transform(X)

In [28]:
X1

array([[ 0.        ,  0.19731918,  0.02945032, ..., -0.64919638,
        -0.41049231, -1.03288546],
       [ 0.        , -0.50974121,  0.23901746, ..., -0.64919638,
        -0.41049231, -1.03288546],
       [ 0.        ,  1.96497014,  0.73435796, ..., -0.64919638,
        -0.41049231, -1.03288546],
       ...,
       [ 0.        , -1.92386197,  0.23901746, ..., -0.64919638,
         2.43609923, -1.03288546],
       [ 0.        , -0.8632714 ,  0.39142992, ..., -0.64919638,
        -0.41049231,  0.96816156],
       [ 0.        , -0.15621101, -0.08485903, ..., -0.64919638,
        -0.41049231, -1.03288546]])

In [29]:
y1=y

In [30]:
y1

,Avg Salary(K)
0,72.0
1,87.5
2,85.0
3,76.5
4,114.5
...,...
737,84.5
738,102.5
739,73.5
740,127.5


In [31]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.2, random_state = 1)
print("The shape of X1_train is:",X1_train.shape)
print("The shape of X1_test is:",X1_test.shape)
print("The shape of y1_train is:",y1_train.shape)
print("The shape of y1_test is:",y1_test.shape)

The shape of X1_train is: (593, 119)
The shape of X1_test is: (149, 119)
The shape of y1_train is: (593, 1)
The shape of y1_test is: (149, 1)


In [32]:
SGD_model = SGDRegressor(loss="squared_loss",alpha = 0.1, max_iter = 3000) #Instantiation
SGD_model.fit(X1_train, y1_train) #Training

C:\Users\OMKAR\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:165: FutureWarning: The loss 'squared_loss' was deprecated in v1.0 and will be removed in version 1.2. Use `loss='squared_error'` which is equivalent.
  warnings.warn(
C:\Users\OMKAR\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDRegressor(alpha=0.1, loss='squared_loss', max_iter=3000)

In [33]:
# predict the values
y1_pred_SGD  = SGD_model.predict(X1_test)

In [34]:
y1_pred_SGD

array([146.17822352,  74.87268239, 121.21060277,  87.79222273,
        82.33411807,  82.22376336,  87.72107629,  67.82260446,
        58.53475719,  66.8734221 ,  93.36042671,  88.61224572,
        63.63293606,  85.35268371, 106.46610445,  58.68089226,
        92.04795286,  57.32119371,  83.80227212, 101.04794339,
       111.52212746,  66.25032892, 135.63841731, 109.32845591,
        70.22158201, 105.22934263,  92.30981266, 115.46232968,
        80.38804527, 144.58428678, 106.19771939,  52.31414675,
       149.28876133,  91.58870248, 115.40691113,  94.46211967,
        49.63090925,  43.14879298, 108.07893855, 131.75073921,
       101.27941156,  50.2445894 , 121.3291105 ,  95.93100851,
        88.77467958,  96.42721772,  57.32119371, 149.81342062,
        94.2973483 ,  65.56496733,  54.74167451, 131.75073921,
        93.3581593 , 120.16358577,  95.19146698, 121.21060277,
        58.06929141, 141.15751924, 116.45933563,  83.98019629,
       136.46343074, 104.23570691,  39.73522942,  48.27

In [35]:
r_squared_SGD = SGD_model.score(X1_train,y1_train)
n = 593 
p = 119
Adj_r_squared_SGD = 1 - (1-r_squared_SGD)*(n-1)/(n-p-1)
rmse_SGD = sqrt(mean_squared_error(y1_test, y1_pred_SGD))

In [36]:
linreg_full_metrics = pd.Series({'Model': "Linear regression with SGD ",
                     'RMSE':rmse_SGD,
                     'R-Squared': r_squared_SGD,
                     'Adj. R-Squared': Adj_r_squared_SGD     
                   })
result_tabulation = result_tabulation.append(linreg_full_metrics, ignore_index = True)
result_tabulation

C:\Users\OMKAR\AppData\Local\Temp/ipykernel_16636/1081402639.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_tabulation = result_tabulation.append(linreg_full_metrics, ignore_index = True)


,Model,RMSE,R-Squared,Adj. R-Squared
0,Linreg full model,28.33744,0.665127,0.582642
1,Linear regression with SGD,26.90895,0.649354,0.561136


# 3. Decision Tree

In [37]:
from sklearn import tree
from sklearn.tree import export_graphviz
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

In [38]:
decision_tree = DecisionTreeRegressor(criterion = 'squared_error', random_state = 10) #Max depth D.Tree gets formed
decision_tree_model = decision_tree.fit(X_train, y_train) #fit() method is defined inside the class 'DecisionTreeClassifier'

In [39]:
y_pred_DT=decision_tree_model.predict(X_test)

In [40]:
y_pred_DT

array([121.        ,  62.5       , 120.        ,  97.5       ,
        70.        , 109.        ,  65.5       ,  85.5       ,
        72.5       ,  81.        ,  71.75      ,  75.5       ,
        41.5       , 123.5       , 164.5       ,  85.5       ,
        93.33333333,  56.5       ,  84.5       ,  93.5       ,
        89.        ,  66.        , 122.        ,  84.        ,
        73.5       ,  85.5       ,  94.5       , 120.        ,
        80.        , 194.        , 107.        ,  54.        ,
       150.5       ,  82.5       , 139.5       , 120.        ,
        43.        ,  53.5       , 107.5       , 221.5       ,
        75.5       ,  49.        ,  68.5       , 109.        ,
        49.        ,  84.        ,  56.5       , 111.        ,
        64.5       ,  50.        ,  44.5       , 221.5       ,
        90.66666667, 113.5       , 111.5       , 120.        ,
        50.        , 164.        , 113.5       , 177.        ,
       134.        ,  95.        ,  61.        ,  48.5 

In [41]:
y_test

,Avg Salary(K)
273,90.0
414,62.5
529,125.0
502,97.5
729,70.0
...,...
247,50.0
341,111.5
298,162.5
132,81.5


In [42]:
r_squared_DT=decision_tree_model.score(X_test,y_test)
n = 593 
p = 119
Adj_r_squared_DT = 1 - (1-r_squared_DT)*(n-1)/(n-p-1)
Adj_r_squared_DT

0.5608620342087651

In [43]:
# Compute RMSE
rmse_DT = sqrt(mean_squared_error(y_test, y_pred_DT))

In [44]:
linreg_full_metrics = pd.Series({'Model': "Decision Tree Model ",
                     'RMSE':rmse_DT,
                     'R-Squared': r_squared_DT,
                     'Adj. R-Squared': Adj_r_squared_DT     
                   })
result_tabulation = result_tabulation.append(linreg_full_metrics, ignore_index = True)
result_tabulation

C:\Users\OMKAR\AppData\Local\Temp/ipykernel_16636/987164885.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_tabulation = result_tabulation.append(linreg_full_metrics, ignore_index = True)


,Model,RMSE,R-Squared,Adj. R-Squared
0,Linreg full model,28.33744,0.665127,0.582642
1,Linear regression with SGD,26.90895,0.649354,0.561136
2,Decision Tree Model,22.072437,0.649135,0.560862


# 4.Decision Tree with pruning

In [45]:
prune = DecisionTreeRegressor(max_depth = 10, max_leaf_nodes = 32 , random_state = 10)
decision_tree_prune = prune.fit(X_train, y_train)

In [46]:
y_pred_DT_prune=decision_tree_prune.predict(X_test)

In [47]:
y_pred_DT_prune

array([139.93939394,  59.05172414,  81.86206897, 104.3880597 ,
        60.7244898 ,  93.12931034,  98.98837209,  93.12931034,
        59.05172414,  60.7244898 ,  89.32      ,  93.12931034,
        81.86206897,  59.05172414, 139.93939394,  81.86206897,
        98.98837209,  60.7244898 ,  98.98837209,  81.86206897,
       104.3880597 ,  59.05172414, 127.70491803,  89.32      ,
        81.86206897, 104.3880597 ,  93.12931034,  81.86206897,
        98.98837209, 191.125     , 104.3880597 ,  59.05172414,
       139.93939394,  90.63461538, 127.70491803,  81.86206897,
        59.05172414,  59.05172414,  98.98837209, 139.93939394,
       104.3880597 ,  59.05172414, 101.5       ,  98.98837209,
        59.05172414,  89.32      ,  60.7244898 ,  60.7244898 ,
        93.12931034,  59.05172414,  59.05172414, 139.93939394,
        98.98837209,  89.32      ,  90.63461538,  81.86206897,
        59.05172414, 127.70491803, 139.93939394, 177.        ,
        98.98837209, 104.3880597 , 101.5       ,  59.05

In [48]:
r_squared_DT_prune=decision_tree_prune.score(X_test,y_test)
n = 593 
p = 119
Adj_r_squared_DT_prune = 1 - (1-r_squared_DT_prune)*(n-1)/(n-p-1)
Adj_r_squared_DT_prune
rmse_DT_prune = sqrt(mean_squared_error(y_test, y_pred_DT_prune))

In [49]:
linreg_full_metrics = pd.Series({'Model': "Decision Tree Model after pruning ",
                     'RMSE':rmse_DT_prune,
                     'R-Squared': r_squared_DT_prune,
                     'Adj. R-Squared': Adj_r_squared_DT_prune     
                   })
result_tabulation = result_tabulation.append(linreg_full_metrics, ignore_index = True)
result_tabulation

C:\Users\OMKAR\AppData\Local\Temp/ipykernel_16636/1696126247.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_tabulation = result_tabulation.append(linreg_full_metrics, ignore_index = True)


,Model,RMSE,R-Squared,Adj. R-Squared
0,Linreg full model,28.33744,0.665127,0.582642
1,Linear regression with SGD,26.90895,0.649354,0.561136
2,Decision Tree Model,22.072437,0.649135,0.560862
3,Decision Tree Model after pruning,26.664283,0.487965,0.359144


# 5.Decision Tree with grid search CV

In [50]:
# tuned_paramaters = [{'criterion': ['squared_error', 'absolute_error'],
#                      'min_samples_split': [10, 20, 30],
#                      'max_depth': [3, 5, 7, 9,12,15,20,25],
#                      'min_samples_leaf': [15, 20, 25, 30, 35],
#                      'max_leaf_nodes': [5, 10, 15, 20, 25]}]

## 5.Rndom forest
## 6.Random Forest using Grid search CV
## 7. Ensemble Techniques
## 8. Ensemble Techniques(Adaboost)